In [5]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import copy
client = pymongo.MongoClient('localhost',27017)
start = datetime(2019,12,1)
end = datetime(2020,4,18)

In [3]:
ps = '''广东
北京
海外
山东
江苏
浙江
四川
湖北
上海
河南
河北
福建
安徽
湖南
重庆
辽宁
陕西
广西
黑龙江
江西
山西
天津
内蒙古
云南
吉林
贵州
新疆
甘肃
海南
香港
宁夏
西藏
台湾
青海
澳门'''.split('\n')
ps

['广东',
 '北京',
 '海外',
 '山东',
 '江苏',
 '浙江',
 '四川',
 '湖北',
 '上海',
 '河南',
 '河北',
 '福建',
 '安徽',
 '湖南',
 '重庆',
 '辽宁',
 '陕西',
 '广西',
 '黑龙江',
 '江西',
 '山西',
 '天津',
 '内蒙古',
 '云南',
 '吉林',
 '贵州',
 '新疆',
 '甘肃',
 '海南',
 '香港',
 '宁夏',
 '西藏',
 '台湾',
 '青海',
 '澳门']

In [4]:
py = '''guangdong
beijing
haiwai
shandong
jiangsu
zhejiang
sichuan
hubei
shanghai
henan
hebei
fujian
anhui
hunan
chongqing
liaoning
shanxi
guangxi
heilongjiang
jiangxi
shanxi2
tianjin
neimenggu
yunnan
jilin
guizhou
xinjiang
gansu
hainan
xianggang
ningxia
xizang
taiwan
qinghai
aomen'''.split('\n')
py

['guangdong',
 'beijing',
 'haiwai',
 'shandong',
 'jiangsu',
 'zhejiang',
 'sichuan',
 'hubei',
 'shanghai',
 'henan',
 'hebei',
 'fujian',
 'anhui',
 'hunan',
 'chongqing',
 'liaoning',
 'shanxi',
 'guangxi',
 'heilongjiang',
 'jiangxi',
 'shanxi2',
 'tianjin',
 'neimenggu',
 'yunnan',
 'jilin',
 'guizhou',
 'xinjiang',
 'gansu',
 'hainan',
 'xianggang',
 'ningxia',
 'xizang',
 'taiwan',
 'qinghai',
 'aomen']

In [5]:
pro_df = pd.DataFrame({'c':ps,'e':py})
pro_df

,c,e
0,广东,guangdong
1,北京,beijing
2,海外,haiwai
3,山东,shandong
4,江苏,jiangsu
5,浙江,zhejiang
6,四川,sichuan
7,湖北,hubei
8,上海,shanghai
9,河南,henan


In [7]:
already_pro = client.Emotion4_Count_Db.collection_names()
run_pro = pro_df[~pro_df['e'].isin(already_pro)]
run_pro

,c,e
0,广东,guangdong
1,北京,beijing
2,海外,haiwai
3,山东,shandong
4,江苏,jiangsu
5,浙江,zhejiang
6,四川,sichuan
7,湖北,hubei
8,上海,shanghai
9,河南,henan


In [8]:
from tqdm import tqdm
co = client.Weibo_Local_Db.comment

for province in run_pro['c'].tolist():
    print(province)
    id_list = [x['_id'] for x in client.Weibo_Local_Db.user.find({'province':province},{})]
    start_one = copy.deepcopy(start)
    insert_co = client.Emotion4_Count_Db[run_pro[run_pro['c'] == province]['e'].tolist()[0]]
    insert_co.ensure_index('datetime',unique=True)
    query = {'comment_usr_id':
             {'$in':id_list}
            }   
    df = pd.DataFrame(co.find(query))
    while start_one<end:
        end_one = start_one+timedelta(days=1)
        sub_df = df[(df['comment_dt']<=end_one)&(df['comment_dt']>start_one)]
        data = {}
        data['全部']=len(sub_df)
        for label in '快乐 乐观 平静 担忧 无助 恐惧 悲伤 愤怒 恐慌 压力 抑郁 焦虑'.split(' '):
            if label in sub_df.columns:
                data[label]=len(sub_df[label].dropna(how='any'))
            else:
                data[label]=0
        data['datetime'] = start_one
        
        #print(province,start_one, end_one, len(df))
        insert_co.insert_one(data)
        start_one += timedelta(days=1)


/home/lowinli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  
  0%|          | 0/12 [00:00<?, ?it/s]

广东 2019-12-01 00:00:00 2019-12-02 00:00:00 531085
广东 2019-12-02 00:00:00 2019-12-03 00:00:00 531085
广东 2019-12-03 00:00:00 2019-12-04 00:00:00 531085
广东 2019-12-04 00:00:00 2019-12-05 00:00:00 531085
广东 2019-12-05 00:00:00 2019-12-06 00:00:00 531085
广东 2019-12-06 00:00:00 2019-12-07 00:00:00 531085
广东 2019-12-07 00:00:00 2019-12-08 00:00:00 531085
广东 2019-12-08 00:00:00 2019-12-09 00:00:00 531085
广东 2019-12-09 00:00:00 2019-12-10 00:00:00 531085
广东 2019-12-10 00:00:00 2019-12-11 00:00:00 531085
广东 2019-12-11 00:00:00 2019-12-12 00:00:00 531085
广东 2019-12-12 00:00:00 2019-12-13 00:00:00 531085
广东 2019-12-13 00:00:00 2019-12-14 00:00:00 531085
广东 2019-12-14 00:00:00 2019-12-15 00:00:00 531085
广东 2019-12-15 00:00:00 2019-12-16 00:00:00 531085
广东 2019-12-16 00:00:00 2019-12-17 00:00:00 531085


100%|██████████| 12/12 [00:00<00:00, 1996.81it/s]


广东 2019-12-17 00:00:00 2019-12-18 00:00:00 531085
广东 2019-12-18 00:00:00 2019-12-19 00:00:00 531085
广东 2019-12-19 00:00:00 2019-12-20 00:00:00 531085
广东 2019-12-20 00:00:00 2019-12-21 00:00:00 531085
广东 2019-12-21 00:00:00 2019-12-22 00:00:00 531085
广东 2019-12-22 00:00:00 2019-12-23 00:00:00 531085
广东 2019-12-23 00:00:00 2019-12-24 00:00:00 531085
广东 2019-12-24 00:00:00 2019-12-25 00:00:00 531085
广东 2019-12-25 00:00:00 2019-12-26 00:00:00 531085
广东 2019-12-26 00:00:00 2019-12-27 00:00:00 531085
广东 2019-12-27 00:00:00 2019-12-28 00:00:00 531085
广东 2019-12-28 00:00:00 2019-12-29 00:00:00 531085


  0%|          | 0/12 [00:00<?, ?it/s]

...

澳门 2020-03-12 00:00:00 2020-03-13 00:00:00 9425
澳门 2020-03-13 00:00:00 2020-03-14 00:00:00 9425
澳门 2020-03-14 00:00:00 2020-03-15 00:00:00 9425
澳门 2020-03-15 00:00:00 2020-03-16 00:00:00 9425
澳门 2020-03-16 00:00:00 2020-03-17 00:00:00 9425
澳门 2020-03-17 00:00:00 2020-03-18 00:00:00 9425
澳门 2020-03-18 00:00:00 2020-03-19 00:00:00 9425
澳门 2020-03-19 00:00:00 2020-03-20 00:00:00 9425
澳门 2020-03-20 00:00:00 2020-03-21 00:00:00 9425
澳门 2020-03-21 00:00:00 2020-03-22 00:00:00 9425
澳门 2020-03-22 00:00:00 2020-03-23 00:00:00 9425
澳门 2020-03-23 00:00:00 2020-03-24 00:00:00 9425
澳门 2020-03-24 00:00:00 2020-03-25 00:00:00 9425
澳门 2020-03-25 00:00:00 2020-03-26 00:00:00 9425
澳门 2020-03-26 00:00:00 2020-03-27 00:00:00 9425


100%|██████████| 12/12 [00:00<00:00, 1616.41it/s]

澳门 2020-03-27 00:00:00 2020-03-28 00:00:00 9425
澳门 2020-03-28 00:00:00 2020-03-29 00:00:00 9425
澳门 2020-03-29 00:00:00 2020-03-30 00:00:00 9425
澳门 2020-03-30 00:00:00 2020-03-31 00:00:00 9425
澳门 2020-03-31 00:00:00 2020-04-01 00:00:00 9425
澳门 2020-04-01 00:00:00 2020-04-02 00:00:00 9425
澳门 2020-04-02 00:00:00 2020-04-03 00:00:00 9425
澳门 2020-04-03 00:00:00 2020-04-04 00:00:00 9425
澳门 2020-04-04 00:00:00 2020-04-05 00:00:00 9425
澳门 2020-04-05 00:00:00 2020-04-06 00:00:00 9425
澳门 2020-04-06 00:00:00 2020-04-07 00:00:00 9425
澳门 2020-04-07 00:00:00 2020-04-08 00:00:00 9425
澳门 2020-04-08 00:00:00 2020-04-09 00:00:00 9425
澳门 2020-04-09 00:00:00 2020-04-10 00:00:00 9425



100%|██████████| 12/12 [00:00<00:00, 2556.07it/s]


澳门 2020-04-10 00:00:00 2020-04-11 00:00:00 9425
澳门 2020-04-11 00:00:00 2020-04-12 00:00:00 9425
澳门 2020-04-12 00:00:00 2020-04-13 00:00:00 9425
澳门 2020-04-13 00:00:00 2020-04-14 00:00:00 9425
澳门 2020-04-14 00:00:00 2020-04-15 00:00:00 9425
澳门 2020-04-15 00:00:00 2020-04-16 00:00:00 9425
澳门 2020-04-16 00:00:00 2020-04-17 00:00:00 9425
澳门 2020-04-17 00:00:00 2020-04-18 00:00:00 9425


In [8]:
from tqdm import tqdm
co = client.Weibo_Local_Db.comment
start_one = copy.deepcopy(start)
insert_co = client.Emotion4_Count_Db['quanbu']
insert_co.ensure_index('datetime',unique=True)

while start_one<end:
    end_one = start_one+timedelta(days=1)
    query = {'comment_dt':{'$gt':start_one,'$lte':end_one}}  
    df = pd.DataFrame(co.find(query))
    
    data = {}
    data['全部']=len(df)
    for label in '快乐 乐观 平静 担忧 无助 恐惧 悲伤 愤怒 恐慌 压力 抑郁 焦虑'.split(' '):
        if label in df.columns:
            data[label]=len(df[label].dropna(how='any'))
        else:
            data[label]=0
    data['datetime'] = start_one

    print(start_one, end_one, len(df))
    insert_co.insert_one(data)
    start_one += timedelta(days=1)

/home/lowinli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  """


2019-12-01 00:00:00 2019-12-02 00:00:00 44372
2019-12-02 00:00:00 2019-12-03 00:00:00 39597
2019-12-03 00:00:00 2019-12-04 00:00:00 38244
2019-12-04 00:00:00 2019-12-05 00:00:00 39541
2019-12-05 00:00:00 2019-12-06 00:00:00 45532
2019-12-06 00:00:00 2019-12-07 00:00:00 36275
2019-12-07 00:00:00 2019-12-08 00:00:00 29521
2019-12-08 00:00:00 2019-12-09 00:00:00 29034
2019-12-09 00:00:00 2019-12-10 00:00:00 35414
2019-12-10 00:00:00 2019-12-11 00:00:00 38808
2019-12-11 00:00:00 2019-12-12 00:00:00 36614
2019-12-12 00:00:00 2019-12-13 00:00:00 32621
2019-12-13 00:00:00 2019-12-14 00:00:00 39356
2019-12-14 00:00:00 2019-12-15 00:00:00 36607
2019-12-15 00:00:00 2019-12-16 00:00:00 38725
2019-12-16 00:00:00 2019-12-17 00:00:00 33175
2019-12-17 00:00:00 2019-12-18 00:00:00 29202
2019-12-18 00:00:00 2019-12-19 00:00:00 30424
2019-12-19 00:00:00 2019-12-20 00:00:00 26836
2019-12-20 00:00:00 2019-12-21 00:00:00 34476
2019-12-21 00:00:00 2019-12-22 00:00:00 32147
2019-12-22 00:00:00 2019-12-23 00: